## this script will parse an ATP file for EMHF V1. This can easily be modified for EMHF V2 at some point.

<parameter name="Initial NVM Offsets" units="deg">-3.1750</parameter>
this how thing is a tag

<ATPTest>
  <TestSteps>
    <Step name="NVM Program." result="Pass" type="NVM Programming">
      <Axis1 result="Pass">
        <parameter name="Initial NVM Offsets" units="deg">-3.1750</parameter>

root = ATPTest
root[0] = <Element 'TestSteps' at 0x000002982872C1D8>
root[0].tag = TestSteps
root[0][0].tag = Step
root[0][0].attrib = {'name': 'NVM Program.', 'result': 'Pass', 'type': 'NVM Programming'}
root[0][0].attrib['name'] = NVM Program
root[0][0][0].tag = Axis1
root[0][0][0][0].text = -3.1750

## methods for finding info:

for attribute in tree.iter():
    print(attribute.tag, attribute.attrib)

tree(path='TestSteps/Step/Axis1/parameter[@name="Initial NVM Offsets"]'):

for elem in tree.iterfind(path='TestSteps/Step/Axis1/parameter[@name="Initial NVM Offsets"]'): #fyi, this is an XPATH
    print(elem.tag, elem.text)

print(root)


## the following methods might be used to get full path info
from os import listdir, getcwd
cwd = getcwd()
print(cwd)
print(listdir(cwd+'\\data'),'\n')
#cwd+'\\data\\'

# development path forward
x- write/append data to a file and close it when finished
x- get a list of files from a directory and open, parse, and close each one in order
x- make each block below a module

In [65]:
import glob
import datetime
xml_list = glob.iglob('**\*.xml', recursive=True)
# xml_list = glob.glob('**\*.xml', recursive=True)

# [i for i in xml_list]

In [66]:
# file operations block
x = datetime.datetime.now()

filename = 'data_' + x.strftime('%Y%m%U%H%M') + '.csv'

# initialize the file with headers
f = open(filename,'w')  # write in text mode
f.write('date_time,serial,axis,test,result,version\n')
f.close()

# open, write, close file
def file_operations(text):
    f = open(filename,'a')  # write in text mode
    f.write(text)
    f.close()
    return

In [67]:
# this function parses all of the data
def data_extraction(root, file_name):
    step = 0
    axis = 0
    parameter = 0

    datetimesec = file_name[-18:-14] + '-' + file_name[-14:-12] + '-' + file_name[-12:-10] + ' ' + file_name[-10:-8] + ':' + file_name[-8:-6] + ':' + file_name[-6:-4]

    try:
        for step in range(15):
            try:
                for axis in range(4):
                    try:
                        for parameter in range(7):
                            data_line = datetimesec + ',' + root[1].text + ',' + root[3][step].attrib['name'] + ',' + root[3][step][axis].tag + ',' + root[3][step][axis][parameter].attrib['name'] + ',' + root[3][step][axis][parameter].text

                            # this section splits the CSV line into a list, appends v1 or v2 to the list, remove v1 or v2 from test name, and puts the string back together
                            x = data_line.split(',')
                            data_line = ','
                            
                            # drop V1 or V2 from the test
                            if x[2][-2::] == 'v1':
                                x.append('v1')
                                x[2] = x[2][:-3]
                            elif x[2][-2::] == 'v2':
                                x.append('v2')
                                x[2] = x[2][:-3]
                            else:
                                x.append('v1')
                                
                            # rename test type correctly
                            if x[2][:4] == 'Back':
                                x[2] = 'Backlash'
                            elif x[2][:3] == 'NVM':
                                x[2] = 'NVM Programming'
                            elif x[2][-3:] == '(+)':
                                x[2] = x[2][:-3:] + '+'
                            elif x[2][-3:] == '(-)':
                                x[2] = x[2][:-3:] + '-'
                            else:
                                pass
                            
                            # rename tests correctly
                            if x[4][:4] == 'Edge':
                                x[4] = x[2]
                            elif x[4][:4] == 'Stal':
                                x[4] = x[2]
                            else:
                                pass
                            
                            # clean up the results
                            if x[5][0] == 'n':
                                x[5] = 'none'
                            
                            x.pop(2)
                            file_operations(data_line.join(x) + '\n')
                    except:
                        continue
            except:
                continue
    except:
        print('loop failed')
    
    return

In [68]:
startTime = datetime.datetime.now()
print(datetime.datetime.now() - startTime)

# this block opens the file, creates an object, and gets the root
import xml.etree.ElementTree as ET
try:
#     for i in xml_list[0:50]:
    for i in xml_list:
        tree = ET.parse(i)
        root = tree.getroot()

        # extract the data
        data_extraction(root, i)
except:
    print('failed to find any data')
print('finished!')
print(datetime.datetime.now() - startTime)

0:00:00
finished!
0:00:26.147734
